In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ---- DL ----
import torch
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, TimeDistributed, RepeatVector, Dense

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils 

class TimeDistributed(nn.Module):
    def __init__(self, module):
        super(TimeDistributed, self).__init__()
        self.module = module

    def forward(self, x):
        if len(x.size()) <= 2:
            return self.module(x)
        x_reshape = x.contiguous().view(-1, x.size(-1)) 
        y = self.module(x_reshape)
        if len(x.size()) == 3:
            y = y.contiguous().view(x.size(0), -1, y.size(-1))
        return y

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.LSTM(
                               input_size = 1, 
                               hidden_size = 16, 
                               dropout = 0.25,
                               num_layers = 2,
                               bias = True,
                               batch_first = True,
                               bidirectional = True,
                               )
        self.decoder = nn.LSTM(
                               input_size = 32, 
                               hidden_size = 16, 
                               dropout = 0.25,
                               num_layers = 2,
                               bias = True,
                               batch_first = True,
                               bidirectional = True,
                               )
        self.fc = TimeDistributed(nn.Linear(32, 1))
                
    def forward(self, x):
        h0, (h_n, c_n) = self.encoder(x)
        h0, (h_n, c_n) = self.decoder(h0[:,-1:,:].repeat(1,5,1))
        out = self.fc(h0)
        return out


torch.nn.Module은 모든 뉴럴 네트워크 모듈의 기본 클래스입니다. 일반적인 모델들은 이 클래스를 상속받아야 합니다. 
모듈들은 다른 모듈을 또 포함할 수 있습니다. 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있습니다.

torch.nn.Module 은 PyTorch의 모든 Neural Network의 Base Class입니다. 
모듈은 다른 모듈을 포함할 수 있고, 트리 구조로 형성할 수 있습니다. 
nn 패키지는 신경망 계층(layer)과 거의 비슷한 Module 의 집합을 정의합니다. 
Module은 입력 텐서를 받고 출력 텐서를 계산하는 한편, 학습 가능한 매개변수를 갖는 텐서들을 내부 상태(internal state)로 갖습니다. 
nn 패키지는 또한 신경망을 학습시킬 때 주로 사용하는 유용한 손실 함수(loss function)들도 정의하고 있습니다.

## data

In [13]:
df = pd.read_csv('./data/SPY.csv',parse_dates=['Date'],index_col=0)
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,112.370003,113.389999,111.510002,113.330002,92.246048,118944600
2010-01-05,113.260002,113.680000,112.849998,113.629997,92.490204,111579900
2010-01-06,113.519997,113.989998,113.430000,113.709999,92.555328,116074400
2010-01-07,113.500000,114.330002,113.180000,114.190002,92.946060,131091100
2010-01-08,113.889999,114.620003,113.660004,114.570000,93.255348,126402800


## creat sequence

In [47]:
def reshape_data(x):
    x = np.array(x)
    if x.ndim == 1:
        reshape = np.array(x).reshape(-1,1)
    if x.ndim == 2:
        reshape = np.array(x).reshape(-1,2)
    return reshape
        

def create_lstm_sequence(X,time_step):
    X_out,y_out = [], []
    for i in range(len(X)-time_step):
        X_out.append([X[i:i+t] for t in range(time_step)])
        y_out.append(X[i+time_step])
        
    X_out = np.array(X_out)
    y_out = np.array(y_out)
    
    return X_out,y_out

In [48]:
reshape = reshape_data(df.Close)
reshape.shape

(2263, 1)

In [49]:
X, y = create_lstm_sequence(reshape,4)

C:\Users\21ckw\AppData\Local\Temp/ipykernel_7148/1320030256.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_out = np.array(X_out).reshape(-1,1,time_step)


ValueError: cannot reshape array of size 2259 into shape (1,4)

In [ ]:
X

In [43]:
print(X.shape, y.shape)

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
model = Sequential()
model.add(LSTM(128,input_size=()))

In [ ]:
model = Sequential(
    [
        Input(shape=(x_train.shape[1], x_train.shape[2])),
        LSTM(
            filters=32, kernel_size=7, padding="same", strides=2, activation="relu"
        ),
        LSTM(
            filters=16, kernel_size=7, padding="same", strides=2, activation="relu"
        )
    ]
)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse")
model.summary()